In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service # Import Service class to specify ChromeDriver location
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
import time
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Replace with the path to your ChromeDriver executable
options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-sh-usage")
# options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get('https://www.amazon.com/')
# Open Facebook login page

In [19]:
df = pd.read_csv('product_links/men/mens_accessories.csv')
links = df['links']
links

0       https://www.amazon.com/sspa/click?ie=UTF8&spc=...
1       https://www.amazon.com/sspa/click?ie=UTF8&spc=...
2       https://www.amazon.com/sspa/click?ie=UTF8&spc=...
3       https://www.amazon.com/Nautica-N2239S-Polarize...
4       https://www.amazon.com/Costa-Del-Mar-Original-...
                              ...                        
2103    https://www.amazon.com/Maui-Jim-PolarizedPlus2...
2104    https://www.amazon.com/Serengeti-Bellemon-Pola...
2105    https://www.amazon.com/Travelon-Blocking-Singl...
2106    https://www.amazon.com/sspa/click?ie=UTF8&spc=...
2107    https://www.amazon.com/Magpul-Tactical-Sunglas...
Name: links, Length: 2108, dtype: object

In [ ]:
reviewers = []

for i, link in enumerate(links):
    if i < 100 and i < len(links):
        print(i)
        driver.get(link)
        time.sleep(1)
        try:
            rating_link = driver.find_element(By.ID, "acrCustomerReviewLink")
            rating_link.click()
            see_more_div = driver.find_element(By.ID, "reviews-medley-footer")
            anchor_tag = see_more_div.find_element(By.TAG_NAME, "a")
            driver.get(anchor_tag.get_attribute("href"))
            count = 0
            while True:
                try:
                    time.sleep(1)
                    review_div = driver.find_element(By.ID, "cm_cr-review_list")
                    reviewer_profiles = review_div.find_elements(By.CLASS_NAME, "a-profile")
                    count += len(reviewer_profiles)
                    for profile in reviewer_profiles:
                        reviewers.append(profile.get_attribute("href"))
                    see_more_link = driver.find_element(By.CLASS_NAME, "a-last")
                    a = see_more_link.find_element(By.TAG_NAME, "a")
                    driver.get(a.get_attribute("href"))

                except:
                    break
            print(count)
            print(driver.title)
            time.sleep(1)
        except:
            print("Failed to load reviews")
            continue
    else:
        break
print("complete")

In [ ]:
len(list(set(reviewers)))

In [18]:
df = pd.DataFrame(columns=['links'])

df['links'] = list(set(reviewers))

df.to_csv('reviewers_links/mens_accessories_reviewers.csv', index=False)